## Video Load

In [2]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

from util import *

In [178]:
def get_ROI_mask(h):
    # H 채널에 대한 히스토그램 구하기
    hist_h = cv2.calcHist([h], [0], None, [256], [0, 256])
    
    # 히스토그램 노이즈 제거
    hist_h = cv2.GaussianBlur(hist_h, (1, 3), sigmaX=0, sigmaY=0)

    # 히스토그램 평균, 분산, 표준 편차 구하기
    hist_h /= hist_h.sum()

    mean, variance_l, variance_r = 0, 0, 0
    
    for i in range(len(hist_h)):
        mean += hist_h[i] * i

    for i in range(len(hist_h)):
        variance = (hist_h[i] * (i - mean) ** 2) * 2

        if (i <= mean):
            variance_l += variance
        else:
            variance_r += variance

    SD_l = math.sqrt(variance_l)
    SD_r = math.sqrt(variance_r)

    # 히스토그램을 이미지로 표시
    hist_img = get_histogram_image(hist_h)

    l_weight, r_weight = 1.35, 0.4

    min_range, max_range = 24, 255

    l, r = int(max(mean - SD_l * l_weight, min_range)), int(min(mean + SD_r * r_weight, max_range))

    draw_line_on_histogram_image(hist_img, l, (0, 0, 255))
    draw_line_on_histogram_image(hist_img, r, (0, 0, 255))
    draw_line_on_histogram_image(hist_img, int(mean), (255, 0, 255))

    cv2.imshow("Histogram H", hist_img) 

    # 분석한 히스토그램을 바탕으로 필드 영역 추출 
    field_feature = cv2.inRange(h, l, r)

    # 필드로부터 플레이어 추출
    player_size = 12
    kernel = np.ones((player_size, player_size))
    field_feature = cv2.erode(field_feature, kernel) 

    player_mask = cv2.bitwise_not(field_feature)

    kernel = np.ones((9, 9))
    field_feature = cv2.dilate(field_feature, kernel)  

    kernel = np.ones((3, 3))
    field_feature = cv2.erode(field_feature, kernel)  

    # 필드 영역 마스크 생성
    field_mask = np.zeros(field_feature.shape, dtype=np.uint8)

    # 외곽선을 찾아서 필드 영역을 단순화
    contours, hierarchy = cv2.findContours(field_feature, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contour_length = []

    for contour in contours:
        contour_length.append(cv2.arcLength(contour, True))

    if len(contour_length) > 0:
        contour = contours[int(np.argmax(contour_length))]

        cv2.drawContours(field_mask, [cv2.convexHull(contour)], -1, 255, -1) 

    # 필드로부터 플레이어 추출
    player_mask = cv2.copyTo(player_mask, field_mask)            

    return field_mask, player_mask

In [4]:
def rotate(src, degree):
    h, w = src.shape[:2]
    center = (w / 2, h / 2)

    rot = cv2.getRotationMatrix2D(center, degree, 1)

    rad = math.radians(degree)
    sin = math.sin(rad)
    cos = math.cos(rad)
    b_w = int((h * abs(sin)) + (w * abs(cos)))
    b_h = int((h * abs(cos)) + (w * abs(sin)))

    rot[0, 2] += ((b_w / 2) - center[0])
    rot[1, 2] += ((b_h / 2) - center[1])

    outImg = cv2.warpAffine(src, rot, (b_w, b_h), flags=cv2.INTER_LINEAR)
    
    return outImg

In [180]:
def load_video(path, ms=25):

    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)

    (run, frame), prev = capture.read(), None # 다음 Frame 읽기
    
    # 영상 사이즈 기준 1920 x 1080
    # 영상 크롭 및 사이즈 조정 (x1/4)
    def init_frame(frame, crop=64):
        return cv2.resize(frame[crop:-crop,:], (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)

    if run: # Frame을 읽은 경우
         prev = frame = init_frame(frame)

    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우 
            frame = init_frame(frame)
                            
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            h, s, v = cv2.split(hsv)

            field_mask, player_mask = get_ROI_mask(h) 

            # 원근 보정을 위해 필드에서 Lane의 특징 추출
            lane_feature = cv2.copyTo(s, field_mask)
            
            lane_feature = cv2.resize(lane_feature, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)

            lane_feature = cv2.equalizeHist(lane_feature)

            #
            kernel = np.array([[-1] * 10 + [1] * 21 + [-1] * 10] * 4) / (10 * 4)

            lane_feature = cv2.filter2D(lane_feature, -1, kernel)

            height, width = lane_feature.shape

            scaler = (width - height) // 2

            # 정사각형으로 이미지 크롭
            lane_feature = lane_feature[:,scaler:-scaler] 
            
            threshold, lane_feature = cv2.threshold(lane_feature, 24, 255, cv2.THRESH_BINARY)  
            
            lane_feature = cv2.resize(lane_feature, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
            
            kernel = np.array([[-1] * 1 + [1] * 3 + [-1] * 1] * 15) / (1 * 15)
            
            cv2.imshow("perspective", lane_feature) 
            test = []
            good = []
            for degree in range(-60, 60, 2):
                pass
                rotated = rotate(kernel, degree)

                filtered = cv2.filter2D(lane_feature, cv2.CV_32F, rotated)
                
                test.append(filtered.sum())

                good.append(filtered)

            test = np.array(test)
            good = np.array(good)
            cv2.imshow("filtered", cv2.hconcat((good / 10 / 32 / 3)))    

            frame = cv2.copyTo(frame, field_mask)
            cv2.imshow("Frame", frame) 
            cv2.imshow("mask", cv2.hconcat((field_mask, player_mask)))

            cv2.waitKey(ms) # Millisecond 단위로 대기

        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        prev = frame

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거

In [181]:
# Video가 저장된 경로 입력
PATH = r"video/soccer0.mp4"

# Video 재생 및 반환 (Numpy Array)
load_video(PATH, ms=20)

KeyboardInterrupt: 